> Nama: Awwala Nisa Kamila


> NIM: 13519208


> Tugas: Aspek Extraction








# Aspect Extraction

## Import Library

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertModel
import spacy

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from transformers import BertTokenizer
from torch.nn.utils.rnn import pad_sequence

import time
import warnings
import re,string,unicodedata

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Read From Drive

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read Data

In [12]:
df = pd.read_csv('drive/MyDrive/Laptop_Train_v2.csv')
df.head()

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12


## Pre-Processing Data

In [13]:
df = df.drop(labels=["id", "from", "to"], axis=1)
df = df[df["polarity"] != "conflict"]
df = df.reset_index()

In [14]:
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

In [15]:
def remove_between_square_brackets(text):
  return re.sub('\[[^]]*\]', '', text)

df["Sentence"]=df["Sentence"].apply(remove_between_square_brackets)
df["Aspect Term"]=df["Aspect Term"].apply(remove_between_square_brackets)

In [16]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

df["Sentence"]=df["Sentence"].apply(remove_special_characters)
df["Aspect Term"]=df["Aspect Term"].apply(remove_special_characters)

In [17]:
stop=set(stopwords.words('english'))

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

df["Sentence"]=df["Sentence"].apply(remove_stopwords)
df["Aspect Term"]=df["Aspect Term"].apply(remove_stopwords)

In [18]:
def tokenize_sentence(text):
  tokens = word_tokenize(text)
  return tokens

df['tokens_aspect']=df["Aspect Term"].apply(tokenize_sentence)
df['tokens']=df["Sentence"].apply(tokenize_sentence)

In [19]:
df.head()

,index,Sentence,Aspect Term,polarity,tokens_aspect,tokens
0,0,charge night skip taking cord good battery life,cord,neutral,[cord],"[charge, night, skip, taking, cord, good, batt..."
1,1,charge night skip taking cord good battery life,battery life,positive,"[battery, life]","[charge, night, skip, taking, cord, good, batt..."
2,2,tech guy said service center 1to1 exchange dir...,service center,negative,"[service, center]","[tech, guy, said, service, center, 1to1, excha..."
3,3,tech guy said service center 1to1 exchange dir...,sales team,negative,"[sales, team]","[tech, guy, said, service, center, 1to1, excha..."
4,4,tech guy said service center 1to1 exchange dir...,tech guy,neutral,"[tech, guy]","[tech, guy, said, service, center, 1to1, excha..."


In [20]:
df_temp = df
df_temp = df_temp.groupby(['Sentence'])['Aspect Term'].apply(','.join).reset_index()
df_temp

,Sentence,Aspect Term
0,1 12 yes issue one month warranty,warranty
1,1 ghz faster processor stock 500gb hard drive,"processor,500gb hard drive"
2,10 hour battery life youre web browsing word e...,"battery life,battery life,gaming,movie playing..."
3,10 hours battery life really something else,battery life
4,10 plus hours battery,battery
...,...,...
1447,would tell technician knew exactly wrong liste...,technician
1448,wouldnt play firstperson shooter mind wanted r...,MS Office
1449,yet needed customer support yet great thing le...,customer support
1450,youre looking something fly massive spreadshee...,game


In [21]:
hasil = []
for i in range(len(df)):
  for j in range(len(df_temp)):
    if (df["Sentence"][i] == df_temp["Sentence"][j]):
      hasil.append(df_temp["Aspect Term"][j])
      break
    else:
      continue

df["all_aspect_term"] = hasil

In [22]:
word = ""
hasil = []
hasil2 = []

for i in range(len(df)):
  for j in range(len(df["all_aspect_term"][i])):
    if (df["all_aspect_term"][i][j] == ","):
      hasil.append(word)
      word = ""
    elif (j == len(df["all_aspect_term"][i])-1):
      word += df["all_aspect_term"][i][j]
      hasil.append(word)
      hasil2.append(hasil)
      word = ""
      hasil = []
    else:
      word += df["all_aspect_term"][i][j]

df["all_aspect_term"] = hasil2

In [23]:
def tags_sentence(index):
  a = df["tokens"][index]
  b = df["all_aspect_term"][index]

  hasil = []
  count = 0
  for i in range(len(a)):
    for j in range(len(b)):
      if (a[i] == b[j] or a[i] in b[j]):
        count += 1
        if (count > 2):
          hasil.append(2)
        else:
          hasil.append(count)
        break
      elif (j == len(b)-1):
        hasil.append(0)
        count = 0
      else:
        continue
  
  return hasil


hasil2 =[]
for i in range(len(df)):
  hasil2.append(tags_sentence(i))

df["tags"] = hasil2

In [24]:
def pols_sentence(index):
  a = df["tokens"][index]
  b = df["tokens_aspect"][index]
  c = df["polarity"][index]
  

  hasil = []
  for i in range(len(a)):
    for j in range(len(b)):
      if (a[i] == b[j] or a[i] in b[j]):
        if(c == "negative"):
          hasil.append(0)
        elif(c == "neutral"):
          hasil.append(1)
        elif(c == "positive"):
          hasil.append(2)
        break
      elif (j == len(b)-1):
        hasil.append(-1)
      else:
        continue
  
  return hasil


hasil2 =[]
for i in range(len(df)):
  hasil2.append(pols_sentence(i))

df["pols"] = hasil2

In [25]:
df.head()

,index,Sentence,Aspect Term,polarity,tokens_aspect,tokens,all_aspect_term,tags,pols
0,0,charge night skip taking cord good battery life,cord,neutral,[cord],"[charge, night, skip, taking, cord, good, batt...","[cord, battery life]","[0, 0, 0, 0, 1, 0, 1, 2]","[-1, -1, -1, -1, 1, -1, -1, -1]"
1,1,charge night skip taking cord good battery life,battery life,positive,"[battery, life]","[charge, night, skip, taking, cord, good, batt...","[cord, battery life]","[0, 0, 0, 0, 1, 0, 1, 2]","[-1, -1, -1, -1, -1, -1, 2, 2]"
2,2,tech guy said service center 1to1 exchange dir...,service center,negative,"[service, center]","[tech, guy, said, service, center, 1to1, excha...","[service center, sales team, tech guy]","[1, 2, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0]","[-1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1,..."
3,3,tech guy said service center 1to1 exchange dir...,sales team,negative,"[sales, team]","[tech, guy, said, service, center, 1to1, excha...","[service center, sales team, tech guy]","[1, 2, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1,..."
4,4,tech guy said service center 1to1 exchange dir...,tech guy,neutral,"[tech, guy]","[tech, guy, said, service, center, 1to1, excha...","[service center, sales team, tech guy]","[1, 2, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0]","[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."


In [26]:
df = df.drop(labels=["index", "Sentence", "Aspect Term", "polarity", "tokens_aspect", "all_aspect_term"], axis=1)
df.head()

,tokens,tags,pols
0,"[charge, night, skip, taking, cord, good, batt...","[0, 0, 0, 0, 1, 0, 1, 2]","[-1, -1, -1, -1, 1, -1, -1, -1]"
1,"[charge, night, skip, taking, cord, good, batt...","[0, 0, 0, 0, 1, 0, 1, 2]","[-1, -1, -1, -1, -1, -1, 2, 2]"
2,"[tech, guy, said, service, center, 1to1, excha...","[1, 2, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0]","[-1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1,..."
3,"[tech, guy, said, service, center, 1to1, excha...","[1, 2, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1,..."
4,"[tech, guy, said, service, center, 1to1, excha...","[1, 2, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0]","[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."


## Split data

In [27]:
df.to_csv(r'coba-test.csv', index=False)

In [28]:
X_train, X_test = train_test_split(df, test_size=0.33, random_state=42)

## Model

### Utils function

In [29]:
def getTime(t):
  minute, second = divmod(t, 60)
  hour, minute = divmod(minute, 60)
  return int(hour), int(minute), int(second)

def load_model(model, path):
  model.load_state_dict(torch.load(path), strict=False)
  return model
    
def save_model(model, name):
  torch.save(model.state_dict(), name)

### Create Model

In [30]:
class bert_ATE(torch.nn.Module):
  def __init__(self, pretrain_model):
    super(bert_ATE, self).__init__()
    self.bert = BertModel.from_pretrained(pretrain_model)
    self.linear = torch.nn.Linear(self.bert.config.hidden_size, 3)
    self.loss_fn = torch.nn.CrossEntropyLoss()

  def forward(self, ids_tensors, tags_tensors, masks_tensors):
    bert_outputs,_ = self.bert(input_ids=ids_tensors, attention_mask=masks_tensors, return_dict=False)
    linear_outputs = self.linear(bert_outputs)

    if tags_tensors is not None:
      tags_tensors = tags_tensors.view(-1)
      linear_outputs = linear_outputs.view(-1,3)
      loss = self.loss_fn(linear_outputs, tags_tensors)
      return loss
    else:
      return linear_outputs

### Set Dataset

In [31]:
class dataset_ATM(Dataset):
  def __init__(self, df, tokenizer):
    self.df = df
    self.tokenizer = tokenizer

  def __getitem__(self, idx):
    tokens, tags, pols = self.df.iloc[idx, :3].values
    
    bert_tokens = []
    bert_tags = []
    bert_pols = []
    for i in range(len(tokens)):
      t = self.tokenizer.tokenize(tokens[i])
      bert_tokens += t
      bert_tags += [int(tags[i])]*len(t)
      bert_pols += [int(pols[i])]*len(t)
    
    bert_ids = self.tokenizer.convert_tokens_to_ids(bert_tokens)

    ids_tensor = torch.tensor(bert_ids)
    tags_tensor = torch.tensor(bert_tags)
    pols_tensor = torch.tensor(bert_pols)

    return bert_tokens, ids_tensor, tags_tensor, pols_tensor

  def __len__(self):
    return len(self.df)

### Train Model

In [32]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pretrain_model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
model_Aspect_Extraction = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_Aspect_Extraction.parameters(), lr=2e-5)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
data_train = dataset_ATM(X_train, tokenizer)
data_test = dataset_ATM(X_test, tokenizer)

In [34]:
def create_mini_batch(samples):
  ids_tensors = [s[1] for s in samples]
  ids_tensors = pad_sequence(ids_tensors, batch_first=True)

  tags_tensors = [s[2] for s in samples]
  tags_tensors = pad_sequence(tags_tensors, batch_first=True)

  pols_tensors = [s[3] for s in samples]
  pols_tensors = pad_sequence(pols_tensors, batch_first=True)
  
  masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
  masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
  
  return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [35]:
train_loader = DataLoader(data_train, batch_size=5, collate_fn=create_mini_batch, shuffle=True)
test_loader = DataLoader(data_test, batch_size=50, collate_fn=create_mini_batch, shuffle=True)

In [36]:
def train_Model_Aspect_Extraction(loader, epochs):
  all_data = len(loader)
  for epoch in range(epochs):
    finish_data = 0
    losses = []
    current_times = []
    correct_predictions = 0
    
    for data in loader:
      t0 = time.time()
      ids_tensors, tags_tensors, _, masks_tensors = data
      ids_tensors = ids_tensors.to(DEVICE)
      tags_tensors = tags_tensors.to(DEVICE)
      masks_tensors = masks_tensors.to(DEVICE)

      loss = model_Aspect_Extraction(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
      losses.append(loss.item())
      loss.backward()
      optimizer_ATE.step()
      optimizer_ATE.zero_grad()

      finish_data += 1
      current_times.append(round(time.time()-t0,3))
      current = np.mean(current_times)
      hour, minute, second = getTime(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
      print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hour:", hour, " minute:", minute," second:", second)         

    save_model(model_Aspect_Extraction, 'bert_ATE.pkl')

### Test Model

In [37]:
def test_Model_Aspect_Extraction(loader):
  pred = []
  trueth = []
  with torch.no_grad():
    for data in loader:
      ids_tensors, tags_tensors, _, masks_tensors = data
      ids_tensors = ids_tensors.to(DEVICE)
      tags_tensors = tags_tensors.to(DEVICE)
      masks_tensors = masks_tensors.to(DEVICE)

      outputs = model_Aspect_Extraction(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

      _, predictions = torch.max(outputs, dim=2)

      pred += list([int(j) for i in predictions for j in i ])
      trueth += list([int(j) for i in tags_tensors for j in i ])

  return trueth, pred

In [45]:
%time train_Model_Aspect_Extraction(train_loader, 3)

epoch: 0  batch: 1 / 310  loss: 0.004126773215830326  hour: 0  minute: 44  second: 11
epoch: 0  batch: 2 / 310  loss: 0.0266099669970572  hour: 0  minute: 38  second: 54
epoch: 0  batch: 3 / 310  loss: 0.02430672664195299  hour: 0  minute: 36  second: 6
epoch: 0  batch: 4 / 310  loss: 0.019985061371698976  hour: 0  minute: 39  second: 10
epoch: 0  batch: 5 / 310  loss: 0.01649079588241875  hour: 0  minute: 42  second: 21
epoch: 0  batch: 6 / 310  loss: 0.02975406222200642  hour: 0  minute: 42  second: 56
epoch: 0  batch: 7 / 310  loss: 0.0256768869834819  hour: 0  minute: 44  second: 47
epoch: 0  batch: 8 / 310  loss: 0.022599601274123415  hour: 0  minute: 44  second: 57
epoch: 0  batch: 9 / 310  loss: 0.022466078685182665  hour: 0  minute: 48  second: 57
epoch: 0  batch: 10 / 310  loss: 0.021365699428133666  hour: 0  minute: 48  second: 3
epoch: 0  batch: 11 / 310  loss: 0.01961246714927256  hour: 0  minute: 51  second: 23
epoch: 0  batch: 12 / 310  loss: 0.018301212733301025  hour: 0

In [46]:
model_Aspect_Extraction = load_model(model_Aspect_Extraction, 'bert_ATE.pkl')

### Evaluation

In [47]:
%time x, y = test_Model_Aspect_Extraction(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: user 1min 17s, sys: 856 ms, total: 1min 18s
Wall time: 1min 18s
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     21387
           1       0.91      0.89      0.90      1589
           2       0.93      0.93      0.93      1354

    accuracy                           0.98     24330
   macro avg       0.95      0.94      0.94     24330
weighted avg       0.98      0.98      0.98     24330



### Predict

In [52]:
def predict_Aspect_Term(sentence, tokenizer):
  word = []
  sentence = remove_between_square_brackets(sentence)
  sentence = remove_special_characters(sentence)
  sentence = remove_stopwords(sentence)
  sentence = tokenize_sentence(sentence)
  word += sentence

  ids = tokenizer.convert_tokens_to_ids(word)
  input_tensor = torch.tensor([ids]).to(DEVICE)

  with torch.no_grad():
    outputs = model_Aspect_Extraction(input_tensor, None, None)
    _, predictions = torch.max(outputs, dim=2)
  predictions = predictions[0].tolist()

  return word, predictions, outputs

In [53]:
def Aspect_Extraction(text):
  terms = []
  word = ""
  word_pieces, predictions, outputs = predict_Aspect_Term(text, tokenizer)

  for i in range(len(predictions)):
    if (predictions[i] == 1):
      if (len(word) != 0):
        terms.append(word.replace(" ##",""))
      word = word_pieces[i]
    if (predictions[i] == 2):
      word += (" " + word_pieces[i])
          
  if (len(word) != 0):
    terms.append(word.replace(" ##",""))
          
  print("Aspect Term:", terms)

In [54]:
model_Aspect_Extraction = load_model(model_Aspect_Extraction, 'bert_ATE.pkl')

In [57]:
sentence = "The graphics are stunning"
Aspect_Extraction(sentence)

Aspect Term: ['graphics']
